In [2]:
import pandas as pd
import datetime
from xlsxwriter.utility import xl_rowcol_to_cell


def create_optimized_excel(filename='案件進度優化範本.xlsx'):
    """
    生成優化後的代書案件進度追蹤表。
    
    Features:
    1. Config頁面：管理銀行清單、案件狀態、類型 (作為下拉選單來源)。
    2. Main頁面：主工作表，包含凍結窗格、資料驗證、條件式格式。
    3. Gantt Chart：自動化甘特圖視覺效果。
    """

    # ---------------------------------------------------------
    # 1. 準備資料 (模擬資料庫結構)
    # ---------------------------------------------------------

    # 銀行設定檔 (將原本的二維矩陣轉為資料庫列表格式)
    bank_config_data = {
        '銀行名稱': ['富邦', '富邦', '台新', '台新', '土銀', '國泰', '兆豐', '兆豐'],
        '分行/窗口': ['北投', '世貿', '營業部', '維席', '民權', '館前', '內科', '雙和'],
        '聯絡人': ['陳先生', '王小姐', '黃小姐', '張先生', '李小姐', '黃小姐', '陶融', '林小姐'],
        '電話': ['0912-345-678', '02-2720-xxxx', '0925-286-705', '02-2389-xxxx', '2562-9801*208', '2311-0609', '02-8751-xxxx', '02-2248-xxxx'],
        'Email': ['jerry@fubon.com', 'wang@fubon.com', 'christy@dbs.com', 'jeff@taishin.com', '105959@landbank.com', 'cathay@mail.com', '010134@megabank.com', '009881@megabank.com']
    }
    df_bank_config = pd.DataFrame(bank_config_data)

    # 選項設定
    status_options = ['簽約', '用印', '完稅', '過戶', '交屋', '結案', '卡關(訴訟)', '卡關(銀行)']
    type_options = ['一般買賣', '房地合一', '繼承', '贈與', '二胎代償']

    # ---------------------------------------------------------
    # 2. 初始化 Excel Writer (使用 xlsxwriter 引擎)
    # ---------------------------------------------------------
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    workbook = writer.book

    # 定義格式 (Styles)
    fmt_header = workbook.add_format({'bold': True, 'bg_color': '#4F81BD',
                                     'font_color': 'white', 'border': 1, 'align': 'center', 'valign': 'vcenter'})
    fmt_date = workbook.add_format({'num_format': 'mm/dd', 'align': 'center'})
    fmt_date_header = workbook.add_format(
        {'num_format': 'mm/dd', 'rotation': 90, 'bold': True, 'align': 'center', 'font_size': 9})
    fmt_text = workbook.add_format({'text_wrap': True, 'valign': 'top'})
    fmt_warning = workbook.add_format(
        {'bg_color': '#FFC7CE', 'font_color': '#9C0006'})  # 紅色警告
    fmt_done = workbook.add_format(
        {'bg_color': '#C6EFCE', 'font_color': '#006100'})    # 綠色完成
    fmt_gantt = workbook.add_format({'bg_color': '#95B3D7'})  # 甘特圖藍條

    # ---------------------------------------------------------
    # 3. 建立 Config 工作表 (隱藏的後台資料庫)
    # ---------------------------------------------------------
    df_bank_config.to_excel(writer, sheet_name='Config', index=False)
    ws_config = writer.sheets['Config']

    # 寫入狀態與類型選項 (在銀行資料右側)
    ws_config.write('G1', '案件狀態清單', fmt_header)
    ws_config.write_column('G2', status_options)
    ws_config.write('H1', '案件類型清單', fmt_header)
    ws_config.write_column('H2', type_options)

    # 隱藏 Config 頁面 (避免誤刪，實務上可設為隱藏)
    # ws_config.hide()

    # ---------------------------------------------------------
    # 4. 建立 Main 工作表 (主要操作介面)
    # ---------------------------------------------------------
    # 定義主表欄位結構
    main_columns = [
        '案件編號', '買方', '賣方', '案件類型', '目前狀態',       # A-E: 基本資料
        '銀行名稱', '分行', '聯絡人', '電話/Email',               # F-I: 銀行資料 (連動)
        '簽約日',                                               # J: 起始
        '預計完稅', '實際完稅',                                  # K-L
        '預計過戶', '實際過戶',                                  # M-N
        '預計交屋', '實際交屋',                                  # O-P: 結束
        '備註/待辦事項'                                          # Q
    ]

    # 建立空的 DataFrame 並寫入標題
    df_main = pd.DataFrame(columns=main_columns)
    df_main.to_excel(writer, sheet_name='承辦進度',
                     index=False, startrow=1)  # 留一行做甘特圖日期
    ws_main = writer.sheets['承辦進度']

    # 寫入甘特圖的時間軸標題 (從 R 欄開始，往後推 30 週)
    start_date = datetime.date.today() - datetime.timedelta(days=30)
    gantt_col_start = len(main_columns)  # R欄 index 為 17
    for i in range(30):
        date_val = start_date + datetime.timedelta(days=i*7)  # 每週一格
        ws_main.write(0, gantt_col_start + i, date_val, fmt_date_header)
        ws_main.set_column(gantt_col_start + i, gantt_col_start + i, 3)  # 欄寬縮小

    # ---------------------------------------------------------
    # 5. 設定資料驗證與公式 (Data Validation & Formulas)
    # ---------------------------------------------------------
    # 設定資料列範圍 (假設做 100 筆)
    data_rows = 100

    # (D欄) 案件類型 - 下拉選單
    ws_main.data_validation(f'D3:D{data_rows}', {
        'validate': 'list',
        'source': '=Config!$H$2:$H$6'
    })

    # (E欄) 目前狀態 - 下拉選單
    ws_main.data_validation(f'E3:E{data_rows}', {
        'validate': 'list',
        'source': '=Config!$G$2:$G$9'
    })

    # (F欄) 銀行名稱 - 下拉選單
    ws_main.data_validation(f'F3:F{data_rows}', {
        'validate': 'list',
        'source': '=Config!$A$2:$A$20'
    })

    # (G-I欄) 銀行資訊連動 (使用 Excel 公式模擬 XLOOKUP)
    # 注意：Python寫入的是公式字串，使用者開啟時會自動計算
    for row in range(2, data_rows):  # row index 2 is Excel row 3
        excel_row = row + 1
        # 找分行
        formula_branch = f'=IFERROR(VLOOKUP(F{excel_row},Config!$A:$E,2,0),"")'
        ws_main.write_formula(row, 6, formula_branch)  # G欄
        # 找聯絡人
        formula_contact = f'=IFERROR(VLOOKUP(F{excel_row},Config!$A:$E,3,0),"")'
        ws_main.write_formula(row, 7, formula_contact)  # H欄
        # 找電話/Email
        formula_info = f'=IFERROR(VLOOKUP(F{excel_row},Config!$A:$E,4,0)&CHAR(10)&VLOOKUP(F{excel_row},Config!$A:$E,5,0),"")'
        ws_main.write_formula(row, 8, formula_info, fmt_text)  # I欄

    # ---------------------------------------------------------
    # 6. 視覺優化與條件式格式 (Visuals)
    # ---------------------------------------------------------

    # 凍結窗格：凍結上 2 列 (標題+甘特日期) 與 左 3 欄 (基本資料)
    ws_main.freeze_panes(2, 3)

    # 設定欄寬
    ws_main.set_column('A:A', 10)  # 編號
    ws_main.set_column('B:C', 12)  # 買賣方
    ws_main.set_column('F:F', 10)  # 銀行
    ws_main.set_column('I:I', 25)  # 聯絡資訊 (寬一點)
    ws_main.set_column('J:P', 10)  # 日期欄
    ws_main.set_column('Q:Q', 30)  # 備註

    # 格式化標題列
    for col_num, value in enumerate(main_columns):
        ws_main.write(1, col_num, value, fmt_header)

    # 條件 1: 完成標記 (實際日期有填寫 -> 綠色)
    # 針對 L, N, P 欄 (實際完稅、實際過戶、實際交屋)
    for col_char in ['L', 'N', 'P']:
        ws_main.conditional_format(f'{col_char}3:{col_char}{data_rows}', {
            'type': 'no_blanks',
            'format': fmt_done
        })

    # 條件 2: 逾期警示 (今天 > 預計日期 AND 實際日期為空 -> 紅色)
    # 假設 O 欄是預計交屋，P 欄是實際交屋
    ws_main.conditional_format(f'O3:O{data_rows}', {
        'type': 'formula',
        'criteria': '=AND(O3<TODAY(), O3<>"", P3="")',  # 邏輯：預計日過了且沒填實際日
        'format': fmt_warning
    })

    # 條件 3: 甘特圖 (進度條)
    # 邏輯：如果上方日期介於 "簽約日(J)" 和 "預計交屋日(O)" 之間，則填色
    gantt_range = f'R3:AZ{data_rows}'  # 假設甘特圖區域
    ws_main.conditional_format(gantt_range, {
        'type': 'formula',
        'criteria': '=AND(R$1>=$J3, R$1<=$O3)',
        'format': fmt_gantt
    })

    # 寫入一筆範例資料，讓使用者知道怎麼填
    example_data = [
        'AA1148099', '柯遠東', '趙志龍', '一般買賣', '完稅',
        '富邦', '', '', '',  # 銀行欄位由公式自動帶出，這裡留空
        '2025-07-19',       # 簽約
        '2025-09-09', None,  # 預計完稅, 實際完稅
        '2025-09-20', None,  # 預計過戶
        '2025-09-25', None,  # 預計交屋
        '稅單好了，待塗銷二胎'
    ]

    # 寫入範例資料並套用日期格式
    for col_idx, value in enumerate(example_data):
        if col_idx in [9, 10, 12, 14]:  # 日期欄位索引
            if value:
                date_obj = datetime.datetime.strptime(value, '%Y-%m-%d')
                ws_main.write_datetime(2, col_idx, date_obj, fmt_date)
        elif col_idx not in [6, 7, 8]:  # 跳過公式欄位
            if value:
                ws_main.write(2, col_idx, value)

    print(f"Excel 檔案已生成: {filename}")
    writer.close()


# 執行函數
if __name__ == "__main__":
    create_optimized_excel()

Excel 檔案已生成: 案件進度優化範本.xlsx
